In [5]:
import numpy as np
from scipy.special import erfc
from scipy.integrate import quad
from scipy.integrate import simpson
from scipy.integrate import fixed_quad

def integral_by_quad(lam_s, lam_t, lam_u, beta):
    """
    주어진 구간 [lam_s, lam_t]에서
    exp(lam - beta^2 * (lam - lam_u)**2)를
    수치적으로 적분하여 결과와 추정 오차를 반환.
    """

    def integrand(lam, lam_u, beta):
      """
      적분에 쓰일 integrand 함수.
      exp(lam - beta^2 * (lam - lam_u)^2)를 반환.
      """
      return np.exp(lam - beta**2 * (lam - lam_u)**2)

    val, err = quad(integrand, lam_s, lam_t, args=(lam_u, beta))
    return val

def integrate_exp_simpson(lam_s, lam_t, lam_u, beta, num_points=200):
    x_vals = np.linspace(lam_s, lam_t, num_points)
    y_vals = np.exp(x_vals - beta**2 * (x_vals - lam_u)**2)
    
    area = simpson(y_vals, x_vals)
    return area

def integrate_exp_fixed_quad(lam_s, lam_t, lam_u, beta, n=64):
    """
    n: 가우스-레장드르 점 개수
    """
    func = lambda lam: np.exp(lam - beta**2 * (lam - lam_u)**2)
    val, _ = fixed_quad(func, lam_s, lam_t, n=n)
    return val

def integral_by_closed(lam_s, lam_t, lam_u, beta):
    """
    [lam_s, lam_t] 구간에서
    exp(lam - beta^2*(lam - lam_u)^2) 적분의 닫힌형 해를 반환한다.
    """
    def log_erf_diff(a, b):
        return np.log(erfc(b)) + np.log(1.0-np.exp(np.log(erfc(a)) - np.log(erfc(b))))

    try:
        h = lam_t - lam_s
        s = 1 /(beta*h)
        r_u = (lam_u - lam_s) / h

        log_prefactor = lam_t + np.log(h) + ((s*h)**2/4 + h*(r_u-1)) + np.log(0.5*np.sqrt(np.pi)*s)
        upper = (r_u + s**2*h/2)/s
        lower = (r_u + s**2*h/2 - 1)/s
        return np.exp(log_prefactor + log_erf_diff(upper, lower))
    except:
        return np.nan

In [6]:
# log_scales : -1 ~ 1 구간 10등분, Goal : -6 ~ 6 구간까지 증가시켜야함
log_scales = np.linspace(-3, 3, 10)
scales = np.exp(log_scales)

# 하이퍼파라미터 설정
lam_min = -5.0778
lam_max = 5.7618            
lam_range = lam_max - lam_min

for NFE in range(5, 31):
    allclose_list = []

    for step in range(NFE):
        h = lam_range / NFE
        betas = 1/(scales*h)
        
        for r_u in [1, 0, -1, -2]:
            # 적분 구간 [lam_s, lam_t] 정의
            lam_s = lam_min + step * h
            lam_t = lam_min + (step+1) * h
            lam_u = lam_s + r_u * h

            # 각각 수치적분과 닫힌형 해로 적분
            inte1 = np.array([integral_by_quad(lam_s, lam_t, lam_u, beta) for beta in betas])
            inte2 = np.array([integrate_exp_simpson(lam_s, lam_t, lam_u, beta) for beta in betas])
            #inte3 = np.array([integral_by_closed(lam_s, lam_t, lam_u, beta) for beta in betas])

            # 두 방식의 적분 결과를 비교
            allclose = np.allclose(inte1, inte2)
            allclose_list.append(allclose)
            
    allclose_list = np.array(allclose_list)
    print('NFE :', NFE, ' result :', 'Pass' if np.prod(allclose_list) else 'Fail')

NFE : 5  result : Pass
NFE : 6  result : Pass
NFE : 7  result : Pass
NFE : 8  result : Pass
NFE : 9  result : Pass
NFE : 10  result : Pass
NFE : 11  result : Pass
NFE : 12  result : Pass
NFE : 13  result : Pass
NFE : 14  result : Pass
NFE : 15  result : Pass
NFE : 16  result : Pass
NFE : 17  result : Pass
NFE : 18  result : Pass
NFE : 19  result : Pass
NFE : 20  result : Pass
NFE : 21  result : Pass
NFE : 22  result : Pass
NFE : 23  result : Pass
NFE : 24  result : Pass
NFE : 25  result : Pass
NFE : 26  result : Pass
NFE : 27  result : Pass
NFE : 28  result : Pass
NFE : 29  result : Pass
NFE : 30  result : Pass


In [1]:
from scipy.integrate import quadrature